In [1]:
import pikepdf
from pdf2image import convert_from_path
from PIL import Image
import io

In [2]:
def compress_pdf(input_pdf_path, output_pdf_path, image_quality=75, resize_factor=0.5):
    # Step 1: Convert PDF to images using pdf2image
    images = convert_from_path(input_pdf_path)
    
    # Step 2: Compress and downsample images using Pillow
    compressed_images = []
    for img in images:
        # Resize the image (downsample)
        img_resized = img.resize(
            (int(img.width * resize_factor), int(img.height * resize_factor)),
            Image.Resampling.LANCZOS
        )
        
        # Compress image (adjust quality)
        with io.BytesIO() as output_stream:
            img_resized.save(output_stream, format="JPEG", quality=image_quality)
            compressed_images.append(output_stream.getvalue())

    # Step 3: Rebuild the PDF with the compressed images
    with pikepdf.open(input_pdf_path) as pdf:
        # Remove old images and replace with compressed ones
        new_pdf = pikepdf.new()
        
        for page, compressed_image in zip(pdf.pages, compressed_images):
            # Create a new page with the compressed image
            page_copy = new_pdf.pages.add()
            page_copy.images['image'] = compressed_image
            new_pdf.pages.append(page_copy)

        # Save the new compressed PDF
        new_pdf.save(output_pdf_path)

    print(f"Compressed PDF saved as {output_pdf_path}")

In [3]:
input_pdf = "icoen.pdf"
output_pdf = "output_compressed.pdf"
compress_pdf(input_pdf, output_pdf)

AttributeError: 'pikepdf._core.PageList' object has no attribute 'add'